In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
from anytree import Node, RenderTree, find, Walker
from collections import OrderedDict 

In [ ]:
# https://towardsdatascience.com/fp-growth-frequent-pattern-generation-in-data-mining-with-python-implementation-244e561ab1c3

In [2]:
class AssociationRule:
    def __init__(self,s,l,sn=0,ln=0):
        self.s = s
        self.l = l
        self.sn = sn
        self.ln = ln

In [3]:
class CPB:
    def __init__(self,path,count):
        self.path = path
        self.count = count

In [4]:
class BasicItem:
    def __init__(self,name,count):
        self.name = name
        self.count = count

In [5]:
class ItemTable:
    def __init__(self,name=None,count=None,idxs=None):
        self.name = name
        self.count = count
        self.idxs = idxs
        self.head = []
        self.CPBase = []
        self.CFPTree = []
        self.CFPTreeTable = []
        self.CFP=[]
        self.FP = []
        self.StrongRule = []

In [15]:
class FPGrowth:
    def __init__(self,minSuppCount=2,minConfidence=0.5):
        self.minSuppCount = minSuppCount
        self.df = None
        self.basicItem = None
        self.rawSet = None
        self.minConfidence = minConfidence
        
    def read_csv(self,file):
        df = pd.read_fwf(file, header=None)
        df = df[0].str.split(',', expand=True)
        self.df = df
        
    def resolveDuplicated(self):
        rawSet = self.df.to_numpy()
        for nRow,row in enumerate(rawSet):
            counter = 1
            for nCol,col in enumerate(row):
                if col == None:
                    continue
                for restN in range(nCol+1,len(row)):
                    if(col == row[restN]):
                        row[restN] = str(col) + str(counter)
                        counter = counter + 1
        self.basicItem = np.unique(rawSet[rawSet != np.array(None)])
        self.rawSet = rawSet.astype('object')
        return self.basicItem
    
    def generateItemTable(self):
        itemTbl = []
        for basicItem in self.basicItem:
#             print(f'===={basicItem}====')
            mask = np.isin(self.rawSet,basicItem)
            vec_mask = np.isin(mask.sum(axis=1), [1])
            idxs = np.where(vec_mask)
#             print(f'{idxs[0]} => {len(idxs[0])}')
            count = len(idxs[0])
            itemTbl.append(ItemTable(basicItem,count,idxs[0]))
        return itemTbl
    
#     def calculateMinSuppCount(self,basicItem):
#         #https://stackoverflow.com/questions/56419519/selecting-rows-of-numpy-array-that-contains-all-the-given-values
#         item = basicItem
        
#         if(type(item) == np.str):
#             k = 1
#         elif(type(item) == np.ndarray):
#             k = len(item)
#         else:
#             k = len(item)
#         mask = np.isin(self.rawSet,item)
#         vec_mask = np.isin(mask.sum(axis=1), [k])
#         ids = np.where(vec_mask)
#         minSuppCount = len(ids[0])
#         return minSuppCount
            
#     def generateItemTable(self,basicItemset):
#         itemTables = []
#         for basicItem in basicItemset:
#             suppCount = self.calculateMinSuppCount(basicItem)
#             itemtable = ItemTable(basicItem,suppCount)
#             itemTables.append(itemtable)
#         return itemTables
    def reverseName(self,itemTables):
        alphaDsc = sorted(itemTables, key=lambda x: x.name,reverse=True)
        return sorted(alphaDsc, key=lambda x: x.count)

    def getAscItemTable(self,itemTables):
#         https://stackoverflow.com/questions/48727337/python-how-to-sort-list-of-object
         return sorted(itemTables, key=lambda x: (x.count))
        
    def getDscItemTable(self,itemTables):
#         https://stackoverflow.com/questions/403421/how-to-sort-a-list-of-objects-based-on-an-attribute-of-the-objects
#         return sorted(itemTables, key=lambda x: x.count,reverse=True)
        alphaAsc = sorted(itemTables, key=lambda x: x.name)
        return sorted(alphaAsc, key=lambda x: x.count,reverse=True)

    def filterItemTable(self,itemTables):
#         https://stackoverflow.com/questions/7623715/deleting-list-elements-based-on-condition
#         return filter(lambda x: x.count >= self.minSuppCount, itemTables)
        return [item for item in itemTables if item.count >= self.minSuppCount]

    def getItemTable(self):
        itemTbl = self.generateItemTable()
#         itemTbl = self.filterItemTable(itemTbl)
        itemTbl = self.getDscItemTable(itemTbl)
        return itemTbl
    
    def displayData(self):
        print(self.df)
        
    def updateItemTable(self,basicItem,currentNode):
        basicItem.head.append(currentNode)
        return basicItem
    
    def updateTree(self,currentNode,basicItem,maxDepth,count=None):
        if count == None:
            count = 1
            
        if currentNode.depth < maxDepth:
            if currentNode.is_leaf:
                newNode = Node(basicItem.name,parent=currentNode,count=count)
                currentNode = newNode
                basicItem = self.updateItemTable(basicItem,currentNode)
            else:
                found = False
                for child in currentNode.children:
                    if child.name == basicItem.name:
                        if count == None:
                            child.count +=1
                        else:
                            child.count += count
                        currentNode = child
                        found=True
                if not found:
                    newNode = Node(basicItem.name,parent=currentNode,count=count)
                    currentNode = newNode
                    basicItem = self.updateItemTable(basicItem,currentNode)
        return currentNode,basicItem
    
    def getFPTree(self,itemTbl):
        root = Node('null')
        currentNode = root
        for i,row in enumerate(self.rawSet):
            currentNode = root
        #     print(f'===={row}====')
            maxDepth = len(row[row != None])
        #     print(f'{maxDepth}')
            for basicItem in itemTbl:
                mask = np.isin(basicItem.idxs,i)
                occurance = mask.sum()
                if(occurance >= 1):
                    currentNode,basicItem = self.updateTree(currentNode,basicItem,maxDepth)
        return root,itemTbl
    
    def filterCPBase(self,itemTbl):
        newItemTbl = []
        for item in itemTbl:
            nodes = item.head
            totalCount = 0
            for node in nodes:
                totalCount += node.count
            if(totalCount >= fp.minSuppCount):
                newItemTbl.append(item)
        return newItemTbl
    
    def generateCPBase(self,itemTbl):
        for item in itemTbl:
            pathList = []
        #     print(f'====={item.name}======')
            nodes = item.head
            for node in nodes:
                paths = []
        #         print('-----')
                for path in node.path:
                    if(path.name != 'null' and path.name!=item.name):
        #                 print(path.name)
                        paths.append(path.name)
                if(len(paths) > 0):
                    pathList.append(CPB(paths,path.count))
            item.CPBase = pathList
        return itemTbl

    def searchBasicItemTable(self,basicItemTable, itemName):
        counter = 0
        for basicItem in basicItemTable:
            if basicItem.name == itemName:
                return counter
            counter+=1
        return -99
    
    def generateCFPTree(self,itemTbl):
        for item in itemTbl:
            root = Node('null')
            currentNode = root
            CFPTreeTable = []
#             print(f'===={item.name}====')
            cpbs = item.CPBase
            basicItemTable = []
            for cpb in cpbs:
                currentNode = root
                maxDepth = len(cpb.path)
#                 print(f'maxDepth=> {maxDepth}')
#                 print(f'{cpb.path}=>{cpb.count}')
                for itemName in cpb.path:
                    index = self.searchBasicItemTable(basicItemTable,itemName)
#                     print(f'{index}  {itemName}')
                    if(index == -99):
                        basicItem = ItemTable(itemName,cpb.count)
                    else:
                        basicItem = basicItemTable[index]

                    currentNode,basicItem = self.updateTree(currentNode,basicItem,maxDepth,cpb.count)

                    if(index == -99):
                        basicItemTable.append(basicItem)
                    else:
                        basicItemTable[index] = basicItem
            item.CFPTreeTable = basicItemTable
            item.CFPTree = root
        return itemTbl
    
    
    
#     def generateCFPTree(self,itemTbl):
#         for item in itemTbl:
#             root = Node('null')
#             currentNode = root
#             CFPTreeTable = []
# #             print(f'===={item.name}====')
#             cpbs = item.CPBase
#             basicItemTable = []
#             for cpb in cpbs:
#                 currentNode = root
#                 maxDepth = len(cpb.path)
# #                 print(f'maxDepth=> {maxDepth}')
# #                 print(f'{cpb.path}=>{cpb.count}')
#                 for itemName in cpb.path:
#                     basicItem = ItemTable(itemName,cpb.count)
#                     currentNode,basicItem = fp.updateTree(currentNode,basicItem,maxDepth,cpb.count)
#                     basicItemTable.append(basicItem)
#             item.CFPTreeTable = basicItemTable
#             item.CFPTree = root
#         return itemTbl
    def pruneBranchTree(self,itemTbl):
        # prune branch that doesnt meet min supprt count
        # https://buildmedia.readthedocs.org/media/pdf/anytree/latest/anytree.pdf
        # 2.3 Detach/Attach Protocol page 8
        # c.parent=None
        for item in itemTbl:
#             print(f'======{item.name}======\n')
#             print(f'---Tree---')
#             print(RenderTree(item.CFPTree))
#             print('\n')
            for node in item.CFPTree.descendants:
#                 currentNode = newTree
        #         print(node)
                if node.count < fp.minSuppCount:
#                     print(f'prune {node.name}')
                    node.parent = None
#             print('\n')
#             print(RenderTree(item.CFPTree))
#             print('===\n')
        return itemTbl

    def removeBranchTable(self,itemTbl):

        # remove the nodeLink if prunned
        # https://stackabuse.com/remove-element-from-an-array-in-python/
        # Using del
        # del array[index]
        for item in itemTbl:
#             print(f'======{item.name}======\n')
#             print(f'---Tree---')
#             print(RenderTree(item.CFPTree))
#             print(f'\n---NodeLink---\n')
            for basicItem in item.CFPTreeTable: 
        #         if(len(basicItem.head) <= 0):
        #             print(f'{basicItem.name} is empty')
        #             continue
#                 print(f'{basicItem.name}=>{basicItem.head}')
                for index,nodeLink in enumerate(basicItem.head):
                    if(nodeLink.root != item.CFPTree.root):
#                         print(f'Remove {nodeLink.name}')
                        del basicItem.head[index]
#                     else:
#                         print(f'Node:{nodeLink}=>{nodeLink.is_leaf}=>{nodeLink.root}')
#                 print('\n')
#             print('===\n')
        return itemTbl

    def generateConditionalPattern(self,itemTbl):
    # generate frequent pattern
        for item in itemTbl:
#             print(f'======{item.name}======\n')
#             print(f'---Tree---')
#             print(RenderTree(item.CFPTree))
#             print(f'\n---NodeLink---\n')
            condPats = []
            for basicItem in item.CFPTreeTable:
                if(len(basicItem.head) <= 0):
#                     print(f'{basicItem.name} is empty')
                    continue
        #         print(f'{basicItem.name}=>{basicItem.head}')
                for index,nodeLink in enumerate(basicItem.head):
                    condPatsItem = {}
#                     print(f'{nodeLink} => {nodeLink.is_leaf} => {nodeLink.root}')
                    currentNode = nodeLink
                    if(not nodeLink.is_leaf):
                        continue
                    while(currentNode.parent):
                        condPatsItem[currentNode.name] = currentNode.count
#                         print(f'Append {currentNode.name} , {currentNode.count}')
                        currentNode = currentNode.parent
#                     print(condPatsItem)
                    if condPatsItem:
                        condPats.append(condPatsItem)
            item.CFP = condPats
#             print(item.FP)
#             print('\n')
        return itemTbl

    # https://www.youtube.com/watch?v=9oPNGofa1pI&t=32s
    def powerset(self,originalSet,maxLength=0):
        subSets = []
        numberOfCombinations = 2 ** len(originalSet)-maxLength
        for combinationIndex in range(1,numberOfCombinations):
            subSet = []
            for setElementIndex in range(0, len(originalSet)):
                if combinationIndex & 1 << setElementIndex:
                    subSet.append(originalSet[setElementIndex])
            subSets.append(subSet)
        return subSets

    def generateFrequentPattern(self,cfp,baseItem):
    #     print('Start generate frequent pattern')
        frequentPatterns = []
        CFPItems = []
    #     print(f'Before reverse: {cfp}')
        cfp = OrderedDict(reversed(list(cfp.items()))) 
    #     print(f'After reverse: {cfp}')
        for key,value in cfp.items():
            CFPItems.append(key)
    #         print(cfp[key])

        subsets = self.powerset(CFPItems)

        for subset in subsets:
            frequentPattern = ((subset+[baseItem],cfp[subset[-1]]))
            frequentPatterns.append(frequentPattern)

    #     print('FrequentPattern')
    #     print(frequentPatterns)
    #     print('End generate frequent pattern')
        return frequentPatterns

    def generateFrequentPatternTable(self,itemTbl):
        # generate frequent pattern table
        for item in itemTbl:
            if(len(item.CFP) <= 0):
                continue
        #     print(f'==={item.name}===')
            frequentPatterns = []
            for cfp in item.CFP:
        #         print(cfp)
                frequentPatterns+=(self.generateFrequentPattern(cfp,item.name))
        #     print(frequentPatterns)
        #     print('----')
            for i,frequentPattern1 in enumerate(frequentPatterns):
                for j in range(i+1,len(frequentPatterns)-1):
                    frequentPattern2 = frequentPatterns[j]
                    if(frequentPattern1[0] == frequentPattern2[0]):
                        newCount = frequentPattern1[1] + frequentPattern2[1]
                        newName = frequentPattern1[0]
                        newTup = (newName,newCount)
                        frequentPatterns[i] = newTup
                        del frequentPatterns[j]
        #                 print(f'{frequentPattern1} same as {frequentPattern2}')
            item.FP = frequentPatterns
        #     print('\n')
        return itemTbl
    
    def calculateMinSuppCount(self,basicItem):
        #https://stackoverflow.com/questions/56419519/selecting-rows-of-numpy-array-that-contains-all-the-given-values
        item = basicItem
        if(type(item) == np.str):
            k = 1
        elif(type(item) == np.ndarray):
            k = len(item)
        else:
            k = len(item)
        mask = np.isin(self.rawSet,item)
        vec_mask = np.isin(mask.sum(axis=1), [k])
        ids = np.where(vec_mask)
        minSuppCount = len(ids[0])
        return minSuppCount
    
    def generateStrongRule(self,itemTbl,isLargestK=True):
        largestK = 0
        if isLargestK:
            for item in itemTbl:
                frequentPatterns=item.FP
                for frequentPattern in frequentPatterns:
                    largestK = max(largestK,len(frequentPattern[0]))
            print(f'Largest k is {largestK}')

        for item in itemTbl:
#             print(f'==={item.name}===')
            frequentPatterns = item.FP
            for frequentPattern in frequentPatterns:
                itemset = frequentPattern[0]
                ln = frequentPattern[1]
                if(len(itemset)!=largestK and isLargestK):
                    continue
#                 print(frequentPattern)
                subsets = self.powerset(itemset,1)
                item
#                 print(f'subsets: {subsets}')
                for subset in subsets:
                    s = subset
                    l = np.setdiff1d(itemset,subset)
                    sn = self.calculateMinSuppCount(s)
#                     print(f'{s}=>{l} confidence = {ln}/{sn} = {ln/sn}')
                    associationRule = AssociationRule(s,l,sn,ln)
                    item.StrongRule.append(associationRule)
        return itemTbl
    
    def displayItemTable(self,itemTbl):
        for item in itemTbl:
            print(f'{item.name}:{item.count}')
    
    def displayFPTree(self,root):
        print(RenderTree(root))
        
    def displayConditionalPatternBase(self,itemTbl):
        for item in itemTbl:
            print(f'==={item.name}===')
            for cpb in item.CPBase:
                print(f'{cpb.path} : {cpb.count}')
                
    def displayConditionalFPTreeTable(self,itemTbl):
        for item in itemTbl:
            print(f'{item.name}',end=':')
            print(item.CFP)
            
    def displayConditionalFPTree(self,itemTbl):            
        for item in itemTbl:
            print(f'==={item.name}===')
            print(RenderTree(item.CFPTree))
            print('\n')
            
    def displayFrequentPatternGenerated(self,itemTbl):      
        for item in itemTbl:
            if(len(item.CFP) <= 0):
                continue
            print(f'{item.name}:',end='')
            for fp in item.FP:
                print(f'{fp}',end=', ')
            print('\n')
            
    def displayAllFrequentPattern(self,itemTbl):
        for item in itemTbl:
            print(f"(['{item.name}'], {item.count})")
            
        for item in itemTbl:
            for frequentPattern in item.FP:
                print(frequentPattern)

    def displayStrongRule(self,itemTbl):
        print(f'Minimum confidence threshold is {self.minConfidence}')
        for item in itemTbl:
            if(len(item.StrongRule) <= 0):
                continue

            print(item.name)
            for strongRule in item.StrongRule:
                s = strongRule.s
                l = strongRule.l
                sn = strongRule.sn
                ln = strongRule.ln
                confidence = ln/sn
                if confidence < self.minConfidence:
                    continue
                print(f'{s}=>{l} confidence = {ln}/{sn} = {confidence}')
                
    def to_df(self,itemTbl):          
#         print('Item|||Conditional Pattern Base|||Conditional FP-tree')
#         maxLen = 0
        data = {}
        data['Item'] = []
        data['CPB'] = []
        data['CFP'] = []
        data['FP'] = []
        for item in itemTbl:
#             print(item.name,end='  |||')
            data['Item'].append(item.name)
            conditionalPatternBase = []
            for cpb in item.CPBase:
#                 msg = f"{{{cpb.path}:{cpb.count}}}"
#                 maxLen = max(maxLen,len(msg))
#                 print(msg,end=', ')
                dicted = (cpb.path,cpb.count)
                conditionalPatternBase.append(dicted)

#             print('|||',end='')
            conditionalFPTree = []
        #     for uniqueItem in item.CFPTreeTable:
        #         print(uniqueItem.name,end=',')
        #         for path in uniqueItem.
            for cfp in item.CFP:
#                 print(cfp,end=',')
                conditionalFPTree.append(cfp)
#             print('\t',end='')
            data['FP'].append(item.FP)
            data['CFP'].append(conditionalFPTree)
            data['CPB'].append(conditionalPatternBase)
#             print('\n')
        df = pd.DataFrame(data)
        return df
    
    def process(self,file):
        self.read_csv(file)
        self.resolveDuplicated()
        print('Transaction:')
        self.displayData()
        print('\n')
        itemTbl = self.getItemTable()
        print('Rearranged Item:')
        self.displayItemTable(itemTbl)
        print('\n')
        fptree,itemTbl = self.getFPTree(itemTbl)
        print('Frequent Pattern Tree:')
        self.displayFPTree(fptree)
        print('\n')
        itemTbl = self.reverseName(itemTbl)
        itemTbl = self.filterCPBase(itemTbl)
        itemTbl = self.generateCPBase(itemTbl)
        print('Conditional Pattern Base:')
        self.displayConditionalPatternBase(itemTbl)
        print('\n')
        itemTbl = self.generateCFPTree(itemTbl)
        print('Conditional FP Tree before prune:')
        self.displayConditionalFPTree(itemTbl)
        print('\n')
        itemTbl = self.pruneBranchTree(itemTbl)
        itemTbl = self.removeBranchTable(itemTbl)
        itemTbl = self.generateConditionalPattern(itemTbl)
        print('Conditional FP Tree Result:')
        self.displayConditionalFPTreeTable(itemTbl)
        print('\n')
        itemTbl = self.generateFrequentPatternTable(itemTbl)
        print('Frequent Pattern Generated:')
        self.displayFrequentPatternGenerated(itemTbl)
        print('\n')
        print('All Frequent Pattern:')
        self.displayAllFrequentPattern(itemTbl)
        print('\n')
        itemTbl = fp.generateStrongRule(itemTbl)
        print('\nStrong Rule:')
        self.displayStrongRule(itemTbl)
        print('========================================================================')
        df = fp.to_df(itemTbl)
        print('Display all in one:')
        print(df)
        return df

In [16]:
fp = FPGrowth(2,0.3)
fp.process('item.csv')

Transaction:
    0   1     2     3
0  I1  I2    I5  None
1  I2  I4  None  None
2  I2  I3  None  None
3  I1  I2    I4  None
4  I1  I3  None  None
5  I2  I3  None  None
6  I1  I3  None  None
7  I1  I2    I3    I5
8  I1  I2    I3  None


Rearranged Item:
I2:7
I1:6
I3:6
I4:2
I5:2


Frequent Pattern Tree:
Node('/null')
├── Node('/null/I2', count=7)
│   ├── Node('/null/I2/I1', count=4)
│   │   ├── Node('/null/I2/I1/I5', count=1)
│   │   ├── Node('/null/I2/I1/I4', count=1)
│   │   └── Node('/null/I2/I1/I3', count=2)
│   │       └── Node('/null/I2/I1/I3/I5', count=1)
│   ├── Node('/null/I2/I4', count=1)
│   └── Node('/null/I2/I3', count=2)
└── Node('/null/I1', count=2)
    └── Node('/null/I1/I3', count=2)


Conditional Pattern Base:
===I5===
['I2', 'I1'] : 1
['I2', 'I1', 'I3'] : 1
===I4===
['I2'] : 1
['I2', 'I1'] : 1
===I3===
['I2'] : 2
['I1'] : 2
['I2', 'I1'] : 2
===I1===
['I2'] : 4
===I2===


Conditional FP Tree before prune:
===I5===
Node('/null')
└── Node('/null/I2', count=2)
    └── Node(

,Item,CPB,CFP,FP
0,I5,"[([I2, I1], 1), ([I2, I1, I3], 1)]","[{'I1': 2, 'I2': 2}]","[([I2, I5], 2), ([I1, I5], 2), ([I2, I1, I5], 2)]"
1,I4,"[([I2], 1), ([I2, I1], 1)]",[{'I2': 2}],"[([I2, I4], 2)]"
2,I3,"[([I2], 2), ([I1], 2), ([I2, I1], 2)]","[{'I1': 2}, {'I1': 2, 'I2': 4}]","[([I1, I3], 4), ([I2, I3], 4), ([I2, I1, I3], 2)]"
3,I1,"[([I2], 4)]",[{'I2': 4}],"[([I2, I1], 4)]"
4,I2,[],[],[]


In [ ]:
def process(self,file):
    self.read_csv(file)
    self.resolveDuplicated()
    print('Transaction:')
    self.displayData()
    print('\n')
    itemTbl = self.getItemTable()
    print('Rearranged Item:')
    self.displayItemTable(itemTbl)
    print('\n')
    fptree,itemTbl = self.getFPTree(itemTbl)
    print('Frequent Pattern Tree:')
    self.displayFPTree(fptree)
    print('\n')
    itemTbl = self.reverseName(itemTbl)
    itemTbl = self.filterCPBase(itemTbl)
    itemTbl = self.generateCPBase(itemTbl)
    print('Conditional Pattern Base:')
    self.displayConditionalPatternBase(itemTbl)
    print('\n')
    itemTbl = self.generateCFPTree(itemTbl)
    print('Conditional FP Tree before prune:')
    self.displayConditionalFPTree(itemTbl)
    print('\n')
    itemTbl = self.pruneBranchTree(itemTbl)
    itemTbl = self.removeBranchTable(itemTbl)
    itemTbl = self.generateConditionalPattern(itemTbl)
    print('Conditional FP Tree Result:')
    self.displayConditionalFPTreeTable(itemTbl)
    print('\n')
    itemTbl = self.generateFrequentPatternTable(itemTbl)
    print('Frequent Pattern Generated:')
    self.displayFrequentPatternGenerated(itemTbl)
    print('\n')
    itemTbl = fp.generateStrongRule(itemTbl)
    print('\nStrong Rule:')
    self.displayStrongRule(itemTbl)
    print('========================================================================')
    df = fp.to_df(itemTbl)
    print('Display all in one:')
    print(df)

In [1069]:
fp = FPGrowth(2,0.3)
fp.read_csv('item.csv')
fp.resolveDuplicated()
print('Transaction:')
fp.displayData()
print('\n')
# itemTbl = fp.generateItemTable(fp.basicItem)
# reverseTbl = fp.getDscItemTable(itemTbl)
# filteredTbl = fp.filterItemTable(reverseTbl)
itemTbl = fp.getItemTable()
print('Rearranged Item:')
fp.displayItemTable(itemTbl)
print('\n')
fptree,itemTbl = fp.getFPTree(itemTbl)
print('Frequent Pattern Tree:')
fp.displayFPTree(fptree)
print('\n')
itemTbl = fp.reverseName(itemTbl)
itemTbl = fp.filterCPBase(itemTbl)
itemTbl = fp.generateCPBase(itemTbl)
print('Conditional Pattern Base:')
fp.displayConditionalPatternBase(itemTbl)
print('\n')
itemTbl = fp.generateCFPTree(itemTbl)
print('Conditional FP Tree before prune:')
fp.displayConditionalFPTree(itemTbl)
print('\n')
itemTbl = fp.pruneBranchTree(itemTbl)
itemTbl = fp.removeBranchTable(itemTbl)
itemTbl = fp.generateConditionalPattern(itemTbl)
print('Conditional FP Tree Result:')
fp.displayConditionalFPTreeTable(itemTbl)
print('\n')
itemTbl = fp.generateFrequentPatternTable(itemTbl)
print('Frequent Pattern Generated:')
fp.displayFrequentPatternGenerated(itemTbl)
print('\n')
itemTbl = fp.generateStrongRule(itemTbl)
print('\nStrong Rule:')
fp.displayStrongRule(itemTbl)
print('========================================================================')
df = fp.to_df(itemTbl)
print('Display all in one:')
print(df)
# for item in itemTbl:
#     print(f'{item.name} => {item.count} => {item.idxs} => {item.head}')

Transaction:
    0   1     2     3
0  I1  I2    I5  None
1  I2  I4  None  None
2  I2  I3  None  None
3  I1  I2    I4  None
4  I1  I3  None  None
5  I2  I3  None  None
6  I1  I3  None  None
7  I1  I2    I3    I5
8  I1  I2    I3  None


Rearranged Item:
I2:7
I1:6
I3:6
I4:2
I5:2


Frequent Pattern Tree:
Node('/null')
├── Node('/null/I2', count=7)
│   ├── Node('/null/I2/I1', count=4)
│   │   ├── Node('/null/I2/I1/I5', count=1)
│   │   ├── Node('/null/I2/I1/I4', count=1)
│   │   └── Node('/null/I2/I1/I3', count=2)
│   │       └── Node('/null/I2/I1/I3/I5', count=1)
│   ├── Node('/null/I2/I4', count=1)
│   └── Node('/null/I2/I3', count=2)
└── Node('/null/I1', count=2)
    └── Node('/null/I1/I3', count=2)


Conditional Pattern Base:
===I5===
['I2', 'I1'] : 1
['I2', 'I1', 'I3'] : 1
===I4===
['I2'] : 1
['I2', 'I1'] : 1
===I3===
['I2'] : 2
['I1'] : 2
['I2', 'I1'] : 2
===I1===
['I2'] : 4
===I2===


Conditional FP Tree before prune:
===I5===
Node('/null')
└── Node('/null/I2', count=2)
    └── Node(

In [1029]:
for item in itemTbl:
    print(f'==={item.name}===')
    print(RenderTree(item.CFPTree))

===I5===
Node('/null')
└── Node('/null/I2', count=2)
    └── Node('/null/I2/I1', count=2)
===I4===
Node('/null')
└── Node('/null/I2', count=2)
===I3===
Node('/null')
├── Node('/null/I2', count=4)
│   └── Node('/null/I2/I1', count=2)
└── Node('/null/I1', count=2)
===I1===
Node('/null')
└── Node('/null/I2', count=4)
===I2===
Node('/null')


In [1021]:
# Display conditional fp tree pattern
for item in itemTbl:
    print(item.CFP)

[{'I1': 2, 'I2': 2}]
[{'I2': 2}]
[{'I1': 2}, {'I1': 2, 'I2': 4}]
[{'I2': 4}]
[]


In [1016]:
# display conditional fp tree
for item in itemTbl:
    print(f'==={item.name}===')
    for cpb in item.CPBase:
        print(f'{cpb.path} : {cpb.count}')

===I5===
['I2', 'I1'] : 1
['I2', 'I1', 'I3'] : 1
===I4===
['I2'] : 1
['I2', 'I1'] : 1
===I3===
['I2'] : 2
['I1'] : 2
['I2', 'I1'] : 2
===I1===
['I2'] : 4
===I2===


In [992]:
for item in itemTbl:
    print(f'{item.name}=>{item.count}=>{item.idxs}=>{item.head}')

I5=>2=>[0 7]=>[Node('/null/I2/I1/I5', count=1), Node('/null/I2/I1/I3/I5', count=1)]
I4=>2=>[1 3]=>[Node('/null/I2/I4', count=1), Node('/null/I2/I1/I4', count=1)]
I3=>6=>[2 4 5 6 7 8]=>[Node('/null/I2/I3', count=2), Node('/null/I1/I3', count=2), Node('/null/I2/I1/I3', count=2)]
I1=>6=>[0 3 4 6 7 8]=>[Node('/null/I2/I1', count=4), Node('/null/I1', count=2)]
I2=>7=>[0 1 2 3 5 7 8]=>[Node('/null/I2', count=7)]


In [963]:
def generateStrongRule(self,itemTbl,isLargestK=True):
    largestK = 0
    if isLargestK:
        for item in itemTbl:
            frequentPatterns=item.FP
            for frequentPattern in frequentPatterns:
                largestK = max(largestK,len(frequentPattern[0]))
        print(f'Largest k is {largestK}')

    for item in itemTbl:
        print(f'==={item.name}===')
        frequentPatterns = item.FP
        for frequentPattern in frequentPatterns:
            itemset = frequentPattern[0]
            ln = frequentPattern[1]
            if(len(itemset)!=largestK and isLargestK):
                continue
            print(frequentPattern)
            subsets = self.powerset(itemset,1)
            item
            print(f'subsets: {subsets}')
            for subset in subsets:
                s = subset
                l = np.setdiff1d(itemset,subset)
                sn = self.calculateMinSuppCount(s)
                print(f'{s}=>{l} confidence = {ln}/{sn} = {ln/sn}')
                associationRule = AssociationRule(s,l,sn,ln)
                item.StrongRule.append(associationRule)
    return itemTbl

In [962]:
def displayStrongRule(self,itemTbl):
    for item in itemTbl:
        if(len(item.StrongRule) <= 0):
            continue

        print(item.name)
        for strongRule in item.StrongRule:
            s = strongRule.s
            l = strongRule.l
            sn = strongRule.sn
            ln = strongRule.ln
            print(f'{s}=>{l} confidence = {ln}/{sn} = {ln/sn}')

In [953]:
largestK = 0
isLargestK = True

if isLargestK:
    for item in itemTbl:
        frequentPatterns=item.FP
        for frequentPattern in frequentPatterns:
            largestK = max(largestK,len(frequentPattern[0]))
    print(f'Largest k is {largestK}')

for item in itemTbl:
    print(f'==={item.name}===')
    frequentPatterns = item.FP
    for frequentPattern in frequentPatterns:
        itemset = frequentPattern[0]
        ln = frequentPattern[1]
        if(len(itemset)!=largestK and isLargestK):
            continue
        print(frequentPattern)
        subsets = fp.powerset(itemset,1)
        item
        print(f'subsets: {subsets}')
        for subset in subsets:
            s = subset
            l = np.setdiff1d(itemset,subset)
            sn = fp.calculateMinSuppCount(s)
            print(f'{s}=>{l} confidence = {ln}/{sn} = {ln/sn}')
            associationRule = AssociationRule(s,l,sn,ln)
            item.StrongRule.append(associationRule)

Largest k is 3
===I5===
(['I2', 'I1', 'I5'], 2)
subsets: [['I2'], ['I1'], ['I2', 'I1'], ['I5'], ['I2', 'I5'], ['I1', 'I5']]
['I2']=>['I1' 'I5'] confidence = 2/7 = 0.2857142857142857
['I1']=>['I2' 'I5'] confidence = 2/6 = 0.3333333333333333
['I2', 'I1']=>['I5'] confidence = 2/4 = 0.5
['I5']=>['I1' 'I2'] confidence = 2/2 = 1.0
['I2', 'I5']=>['I1'] confidence = 2/2 = 1.0
['I1', 'I5']=>['I2'] confidence = 2/2 = 1.0
===I4===
===I3===
(['I2', 'I1', 'I3'], 2)
subsets: [['I2'], ['I1'], ['I2', 'I1'], ['I3'], ['I2', 'I3'], ['I1', 'I3']]
['I2']=>['I1' 'I3'] confidence = 2/7 = 0.2857142857142857
['I1']=>['I2' 'I3'] confidence = 2/6 = 0.3333333333333333
['I2', 'I1']=>['I3'] confidence = 2/4 = 0.5
['I3']=>['I1' 'I2'] confidence = 2/6 = 0.3333333333333333
['I2', 'I3']=>['I1'] confidence = 2/4 = 0.5
['I1', 'I3']=>['I2'] confidence = 2/4 = 0.5
===I1===
===I2===


In [959]:
for item in itemTbl:
    if(len(item.StrongRule) <= 0):
        continue
        
    print(item.name)
    for strongRule in item.StrongRule:
        s = strongRule.s
        l = strongRule.l
        sn = strongRule.sn
        ln = strongRule.ln
        print(f'{s}=>{l} confidence = {ln}/{sn} = {ln/sn}')

I5
['I2']=>['I1' 'I5'] confidence = 2/7 = 0.2857142857142857
['I1']=>['I2' 'I5'] confidence = 2/6 = 0.3333333333333333
['I2', 'I1']=>['I5'] confidence = 2/4 = 0.5
['I5']=>['I1' 'I2'] confidence = 2/2 = 1.0
['I2', 'I5']=>['I1'] confidence = 2/2 = 1.0
['I1', 'I5']=>['I2'] confidence = 2/2 = 1.0
I3
['I2']=>['I1' 'I3'] confidence = 2/7 = 0.2857142857142857
['I1']=>['I2' 'I3'] confidence = 2/6 = 0.3333333333333333
['I2', 'I1']=>['I3'] confidence = 2/4 = 0.5
['I3']=>['I1' 'I2'] confidence = 2/6 = 0.3333333333333333
['I2', 'I3']=>['I1'] confidence = 2/4 = 0.5
['I1', 'I3']=>['I2'] confidence = 2/4 = 0.5


In [850]:
# https://www.youtube.com/watch?v=9oPNGofa1pI&t=32s
def powerset(self,originalSet):
    subSets = []
    numberOfCombinations = 2 ** len(originalSet)
    for combinationIndex in range(1,numberOfCombinations):
        subSet = []
        for setElementIndex in range(0, len(originalSet)):
            if combinationIndex & 1 << setElementIndex:
                subSet.append(originalSet[setElementIndex])
        subSets.append(subSet)
    return subSets    

In [849]:
from collections import OrderedDict 
def generateFrequentPattern(self,cfp,baseItem):
#     print('Start generate frequent pattern')
    frequentPatterns = []
    CFPItems = []
#     print(f'Before reverse: {cfp}')
    cfp = OrderedDict(reversed(list(cfp.items()))) 
#     print(f'After reverse: {cfp}')
    for key,value in cfp.items():
        CFPItems.append(key)
#         print(cfp[key])
        
    subsets = self.powerset(CFPItems)
    
    for subset in subsets:
        frequentPattern = ((subset+[baseItem],cfp[subset[-1]]))
        frequentPatterns.append(frequentPattern)
        
#     print('FrequentPattern')
#     print(frequentPatterns)
#     print('End generate frequent pattern')
    return frequentPatterns

In [871]:
# https://www.youtube.com/watch?v=9oPNGofa1pI&t=32s
def powerset_FrequentPatternTable(originalSet):
    subSets = []
    numberOfCombinations = 2 ** len(originalSet)
    for combinationIndex in range(1,numberOfCombinations):
        subSet = []
        for setElementIndex in range(0, len(originalSet)):
            if combinationIndex & 1 << setElementIndex:
                subSet.append(originalSet[setElementIndex])
        subSets.append(subSet)
    return subSets    

In [805]:
from collections import OrderedDict 
def generateFrequentPattern(cfp,baseItem):
#     print('Start generate frequent pattern')
    frequentPatterns = []
    CFPItems = []
#     print(f'Before reverse: {cfp}')
    cfp = OrderedDict(reversed(list(cfp.items()))) 
#     print(f'After reverse: {cfp}')
    for key,value in cfp.items():
        CFPItems.append(key)
#         print(cfp[key])
        
    subsets = powerset(CFPItems)
    
    for subset in subsets:
        frequentPattern = ((subset+[baseItem],cfp[subset[-1]]))
        frequentPatterns.append(frequentPattern)
        
#     print('FrequentPattern')
#     print(frequentPatterns)
#     print('End generate frequent pattern')
    return frequentPatterns

In [853]:
def generateFrequentPatternTable(self,itemTbl):
    # generate frequent pattern table
    for item in itemTbl:
        if(len(item.CFP) <= 0):
            continue
    #     print(f'==={item.name}===')
        frequentPatterns = []
        for cfp in item.CFP:
    #         print(cfp)
            frequentPatterns+=(self.generateFrequentPattern(cfp,item.name))
    #     print(frequentPatterns)
    #     print('----')
        for i,frequentPattern1 in enumerate(frequentPatterns):
            for j in range(i+1,len(frequentPatterns)-1):
                frequentPattern2 = frequentPatterns[j]
                if(frequentPattern1[0] == frequentPattern2[0]):
                    newCount = frequentPattern1[1] + frequentPattern2[1]
                    newName = frequentPattern1[0]
                    newTup = (newName,newCount)
                    frequentPatterns[i] = newTup
                    del frequentPatterns[j]
    #                 print(f'{frequentPattern1} same as {frequentPattern2}')
        item.FP = frequentPatterns
    #     print('\n')
    return itemTbl

In [ ]:
# generate frequent pattern table
for item in itemTbl:
    if(len(item.CFP) <= 0):
        continue
#     print(f'==={item.name}===')
    frequentPatterns = []
    for cfp in item.CFP:
#         print(cfp)
        frequentPatterns+=(generateFrequentPattern(cfp,item.name))
#     print(frequentPatterns)
#     print('----')
    for i,frequentPattern1 in enumerate(frequentPatterns):
        for j in range(i+1,len(frequentPatterns)-1):
            frequentPattern2 = frequentPatterns[j]
            if(frequentPattern1[0] == frequentPattern2[0]):
                newCount = frequentPattern1[1] + frequentPattern2[1]
                newName = frequentPattern1[0]
                newTup = (newName,newCount)
                frequentPatterns[i] = newTup
                del frequentPatterns[j]
#                 print(f'{frequentPattern1} same as {frequentPattern2}')
    item.FP = frequentPatterns
    print('\n')

In [806]:
testDict = itemTbl[0].CFP[0]
print(testDict)

{'I1': 2, 'I2': 2}


In [1048]:
for item in itemTbl:
    if(len(item.CFP) <= 0):
        continue
    print(f'{item.name}:',end='')
    for fp in item.FP:
        print(f'{fp}',end=', ')
    print('\n')

I5:(['I2', 'I5'], 2), (['I1', 'I5'], 2), (['I2', 'I1', 'I5'], 2), 

I4:(['I2', 'I4'], 2), 

I3:(['I1', 'I3'], 4), (['I2', 'I3'], 4), (['I2', 'I1', 'I3'], 2), 

I1:(['I2', 'I1'], 4), 



In [1060]:
print('Item|||Conditional Pattern Base|||Conditional FP-tree')
maxLen = 0
data ={}
data['Item'] = []
data['CPB'] = []
data['CFP'] = []
data['FP'] = []
for item in itemTbl:
    print(item.name,end='  |||')
    data['Item'].append(item.name)
    conditionalPatternBase = []
    for cpb in item.CPBase:
        msg = f"{{{cpb.path}:{cpb.count}}}"
        maxLen = max(maxLen,len(msg))
        print(msg,end=', ')
        dicted = (cpb.path,cpb.count)
        conditionalPatternBase.append(dicted)
        
    print('|||',end='')
    conditionalFPTree = []
#     for uniqueItem in item.CFPTreeTable:
#         print(uniqueItem.name,end=',')
#         for path in uniqueItem.
    for cfp in item.CFP:
        print(cfp,end=',')
        conditionalFPTree.append(cfp)
    print('\t',end='')
    data['FP'].append(item.FP)
    data['CFP'].append(conditionalFPTree)
    data['CPB'].append(conditionalPatternBase)
    print('\n')

Item|||Conditional Pattern Base|||Conditional FP-tree
I5  |||{['I2', 'I1']:1}, {['I2', 'I1', 'I3']:1}, |||{'I1': 2, 'I2': 2},	

I4  |||{['I2']:1}, {['I2', 'I1']:1}, |||{'I2': 2},	

I3  |||{['I2']:2}, {['I1']:2}, {['I2', 'I1']:2}, |||{'I1': 2},{'I1': 2, 'I2': 4},	

I1  |||{['I2']:4}, |||{'I2': 4},	

I2  ||||||	



In [1061]:
df = pd.DataFrame(data)
print(df)
# df.to_csv('test.csv')

  Item                                    CPB  \
0   I5     [([I2, I1], 1), ([I2, I1, I3], 1)]   
1   I4             [([I2], 1), ([I2, I1], 1)]   
2   I3  [([I2], 2), ([I1], 2), ([I2, I1], 2)]   
3   I1                            [([I2], 4)]   
4   I2                                     []   

                               CFP  \
0             [{'I1': 2, 'I2': 2}]   
1                      [{'I2': 2}]   
2  [{'I1': 2}, {'I1': 2, 'I2': 4}]   
3                      [{'I2': 4}]   
4                               []   

                                                  FP  
0  [([I2, I5], 2), ([I1, I5], 2), ([I2, I1, I5], 2)]  
1                                    [([I2, I4], 2)]  
2  [([I1, I3], 4), ([I2, I3], 4), ([I2, I1, I3], 2)]  
3                                    [([I2, I1], 4)]  
4                                                 []  


In [268]:
def searchBasicItemTable(basicItemTable, itemName):
    counter = 0
    for basicItem in basicItemTable:
        if basicItem.name == itemName:
            return counter
        counter+=1
    return -99

In [356]:
# https://www.softwaretestinghelp.com/fp-growth-algorithm-data-mining/
for item in itemTbl:
    root = Node('null')
    currentNode = root
    CFPTreeTable = []
    print(f'===={item.name}====')
    cpbs = item.CPBase
    basicItemTable = []
    for cpb in cpbs:
        currentNode = root
        maxDepth = len(cpb.path)
        print(f'maxDepth=> {maxDepth}')
        print(f'{cpb.path}=>{cpb.count}')
        for itemName in cpb.path:
            index = searchBasicItemTable(basicItemTable,itemName)
            print(f'{index}  {itemName}')
            if(index == -99):
                basicItem = ItemTable(itemName,cpb.count)
            else:
                basicItem = basicItemTable[index]
                
            currentNode,basicItem = fp.updateTree(currentNode,basicItem,maxDepth,cpb.count)
            
            if(index == -99):
                basicItemTable.append(basicItem)
            else:
                basicItemTable[index] = basicItem
    item.CFPTreeTable = basicItemTable
    item.CFPTree = root

====I5====
====I4====
====I3====
====I1====
====I2====


In [279]:
# prune branch that doesnt meet min supprt count
# https://buildmedia.readthedocs.org/media/pdf/anytree/latest/anytree.pdf
# 2.3 Detach/Attach Protocol page 8
# c.parent=None
for item in itemTbl:
    print(f'======{item.name}======\n')
    print(f'---Tree---')
    print(RenderTree(item.CFPTree))
    print('\n')
    for node in item.CFPTree.descendants:
        currentNode = newTree
#         print(node)
        if node.count < fp.minSuppCount:
            print(f'prune {node.name}')
            node.parent = None
    print('\n')
    print(RenderTree(item.CFPTree))
    print('===\n')

======I5======

---Tree---
Node('/null')
└── Node('/null/I2', count=2)
    └── Node('/null/I2/I1', count=2)




Node('/null')
└── Node('/null/I2', count=2)
    └── Node('/null/I2/I1', count=2)
===

======I4======

---Tree---
Node('/null')
└── Node('/null/I2', count=2)




Node('/null')
└── Node('/null/I2', count=2)
===

======I3======

---Tree---
Node('/null')
├── Node('/null/I2', count=4)
│   └── Node('/null/I2/I1', count=2)
└── Node('/null/I1', count=2)




Node('/null')
├── Node('/null/I2', count=4)
│   └── Node('/null/I2/I1', count=2)
└── Node('/null/I1', count=2)
===

======I1======

---Tree---
Node('/null')
└── Node('/null/I2', count=4)




Node('/null')
└── Node('/null/I2', count=4)
===

======I2======

---Tree---
Node('/null')




Node('/null')
===



In [308]:
# remove the nodeLink if prunned
# https://stackabuse.com/remove-element-from-an-array-in-python/
# Using del
# del array[index]
for item in itemTbl:
    print(f'======{item.name}======\n')
    print(f'---Tree---')
    print(RenderTree(item.CFPTree))
    print(f'\n---NodeLink---\n')
    for basicItem in item.CFPTreeTable: 
#         if(len(basicItem.head) <= 0):
#             print(f'{basicItem.name} is empty')
#             continue
        print(f'{basicItem.name}=>{basicItem.head}')
        for index,nodeLink in enumerate(basicItem.head):
            if(nodeLink.root != item.CFPTree.root):
                print(f'Remove {nodeLink.name}')
                del basicItem.head[index]
            else:
                print(f'Node:{nodeLink}=>{nodeLink.is_leaf}=>{nodeLink.root}')
        print('\n')
    print('===\n')

======I5======

---Tree---
Node('/null')
└── Node('/null/I2', count=2)
    └── Node('/null/I2/I1', count=2)

---NodeLink---

I2=>[Node('/null/I2', count=2)]
Node:Node('/null/I2', count=2)=>False=>Node('/null')


I1=>[Node('/null/I2/I1', count=2)]
Node:Node('/null/I2/I1', count=2)=>True=>Node('/null')


I3=>[]


===

======I4======

---Tree---
Node('/null')
└── Node('/null/I2', count=2)

---NodeLink---

I2=>[Node('/null/I2', count=2)]
Node:Node('/null/I2', count=2)=>True=>Node('/null')


I1=>[]


===

======I3======

---Tree---
Node('/null')
├── Node('/null/I2', count=4)
│   └── Node('/null/I2/I1', count=2)
└── Node('/null/I1', count=2)

---NodeLink---

I2=>[Node('/null/I2', count=4)]
Node:Node('/null/I2', count=4)=>False=>Node('/null')


I1=>[Node('/null/I1', count=2), Node('/null/I2/I1', count=2)]
Node:Node('/null/I1', count=2)=>True=>Node('/null')
Node:Node('/null/I2/I1', count=2)=>True=>Node('/null')


===

======I1======

---Tree---
Node('/null')
└── Node('/null/I2', count=4)

---N

In [501]:
# generate frequent pattern
for item in itemTbl:
    print(f'======{item.name}======\n')
    print(f'---Tree---')
    print(RenderTree(item.CFPTree))
    print(f'\n---NodeLink---\n')
    condPats = []
    for basicItem in item.CFPTreeTable:
        if(len(basicItem.head) <= 0):
            print(f'{basicItem.name} is empty')
            continue
#         print(f'{basicItem.name}=>{basicItem.head}')
        for index,nodeLink in enumerate(basicItem.head):
            condPatsItem = {}
            print(f'{nodeLink} => {nodeLink.is_leaf} => {nodeLink.root}')
            currentNode = nodeLink
            if(not nodeLink.is_leaf):
                continue
            while(currentNode.parent):
                condPatsItem[currentNode.name] = currentNode.count
                print(f'Append {currentNode.name} , {currentNode.count}')
                currentNode = currentNode.parent
            print(condPatsItem)
            if condPatsItem:
                condPats.append(condPatsItem)
    item.CFP = condPats
    print(item.CFP)
    print('\n')

======I5======

---Tree---
Node('/null')
└── Node('/null/I2', count=2)
    └── Node('/null/I2/I1', count=2)

---NodeLink---

Node('/null/I2', count=2) => False => Node('/null')
Node('/null/I2/I1', count=2) => True => Node('/null')
Append I1 , 2
Append I2 , 2
{'I1': 2, 'I2': 2}
I3 is empty
[{'I1': 2, 'I2': 2}]


======I4======

---Tree---
Node('/null')
└── Node('/null/I2', count=2)

---NodeLink---

Node('/null/I2', count=2) => True => Node('/null')
Append I2 , 2
{'I2': 2}
I1 is empty
[{'I2': 2}]


======I3======

---Tree---
Node('/null')
├── Node('/null/I2', count=4)
│   └── Node('/null/I2/I1', count=2)
└── Node('/null/I1', count=2)

---NodeLink---

Node('/null/I2', count=4) => False => Node('/null')
Node('/null/I1', count=2) => True => Node('/null')
Append I1 , 2
{'I1': 2}
Node('/null/I2/I1', count=2) => True => Node('/null')
Append I1 , 2
Append I2 , 4
{'I1': 2, 'I2': 4}
[{'I1': 2}, {'I1': 2, 'I2': 4}]


======I1======

---Tree---
Node('/null')
└── Node('/null/I2', count=4)

---NodeLi

In [250]:
for item in itemTbl:
    print(f'======{item.name}======\n')
    print(f'---Tree---')
    print('\n')
    print(RenderTree(item.CFPTree))
    for basicItem in item.CFPTreeTable:
        print(f'{basicItem.name}=>{basicItem.head}')  
    print('===\n')

======I5======

---Tree---


Node('/null')
└── Node('/null/I2', count=2)
    └── Node('/null/I2/I1', count=2)
I2=>[Node('/null/I2', count=2)]
I1=>[Node('/null/I2/I1', count=2)]
I3=>[]
===

======I4======

---Tree---


Node('/null')
└── Node('/null/I2', count=2)
I2=>[Node('/null/I2', count=2)]
I1=>[]
===

======I3======

---Tree---


Node('/null')
├── Node('/null/I2', count=4)
│   └── Node('/null/I2/I1', count=2)
└── Node('/null/I1', count=2)
I2=>[Node('/null/I2', count=4)]
I1=>[Node('/null/I1', count=2), Node('/null/I2/I1', count=2)]
===

======I1======

---Tree---


Node('/null')
└── Node('/null/I2', count=4)
I2=>[Node('/null/I2', count=4)]
===

======I2======

---Tree---


Node('/null')
===



In [638]:
def searchBasicItem(basicItemTable,name):
    for i,basicItem in enumerate(basicItemTable):
        if basicItem.name == name:
            return i
    return False

In [631]:
for basicItem in itemTbl[0].CFPTree:
    print(f'{basicItem.name}=>{basicItem.count}')

TypeError: 'Node' object is not iterable

In [556]:
for item in itemTbl:
    pathList = []
#     print(f'====={item.name}======')
    nodes = item.head
    for node in nodes:
        paths = []
#         print('-----')
        for path in node.path:
            if(path.name != 'null' and path.name!=item.name):
#                 print(path.name)
                paths.append(path.name)
        if(len(paths) > 0):
            pathList.append(CPB(paths,path.count))
    item.CPBase = pathList

In [558]:
for item in itemTbl:
    cpbs = item.CPBase
    for cpb in cpbs:
        print(cpb.path)
        for tup in cpb.path:
            print(tup)

['I2', 'I1']
I2
I1
['I2', 'I1', 'I3']
I2
I1
I3
['I2']
I2
['I2', 'I1']
I2
I1
['I2']
I2
['I1']
I1
['I2', 'I1']
I2
I1
['I2']
I2


In [559]:
def updateCFPTree(currentNode,itemName,count,maxDepth):
    if currentNode.depth < maxDepth:
        if currentNode.is_leaf:
            newNode = Node(itemName,parent=currentNode,count=count)
            currentNode = newNode
        else:
            found = False
            for child in currentNode.children:
                if child.name == itemName:
                    child.count += count
                    currentNode = child
                    found=True
            if not found:
                newNode = Node(itemName,parent=currentNode,count=count)
                currentNode = newNode
    return currentNode

In [553]:
for item in itemTbl:
    root = Node('null')
    currentNode = root
    print(f'===={item.name}====')
    cpbs = item.CPBase
    for cpb in cpbs:
        currentNode = root
        maxDepth = len(cpb.path)
        print(f'maxDepth=> {maxDepth}')
        print(f'{cpb.path}=>{cpb.count}')
        for itemName in cpb.path:
            print(itemName)
            currentNode = updateCFPTree(currentNode,itemName,cpb.count,maxDepth)
    item.CFPTreeHead = root

====I5====
maxDepth=> 2
['I2', 'I1']=>1
I2
I1
maxDepth=> 3
['I2', 'I1', 'I3']=>1
I2
I1
I3
====I4====
maxDepth=> 1
['I2']=>1
I2
maxDepth=> 2
['I2', 'I1']=>1
I2
I1
====I3====
maxDepth=> 1
['I2']=>2
I2
maxDepth=> 1
['I1']=>2
I1
maxDepth=> 2
['I2', 'I1']=>2
I2
I1
====I1====
maxDepth=> 1
['I2']=>4
I2
====I2====


In [554]:
print(itemTbl[0].name)
print(RenderTree(itemTbl[0].CFPTreeHead))

I5
Node('/null')
└── Node('/null/I2', count=2)
    └── Node('/null/I2/I1', count=2)
        └── Node('/null/I2/I1/I3', count=1)


In [772]:
# root = Node('null')
# currentNode = root
# for i,row in enumerate(fp.rawSet):
#     currentNode = root
# #     print(f'===={row}====')
#     maxDepth = len(row[row != None])
# #     print(f'{maxDepth}')
#     for basicItem in itemTbl:
#         mask = np.isin(row,basicItem.name)
#         occurance = mask.sum()
#         if(occurance >= 1):
#             if basicItem.name in currentNode.children:
#                 for child in currentNode.children:
#                     if child.name == basicItem.name:
#                         child.count +=1
#                         currentNode = child
#             else:
#                 newNode = Node(basicItem.name,parent=currentNode,count=1)
#                 currentNode = newNode
                
# print(RenderTree(root))

In [764]:
fp = FPGrowth(2)
fp.read_csv('item.csv')
fp.resolveDuplicated()
# itemTbl = fp.generateItemTable(fp.basicItem)
# reverseTbl = fp.getDscItemTable(itemTbl)
# filteredTbl = fp.filterItemTable(reverseTbl)
itemTbl = fp.generateItemTable()
itemTbl = fp.filterItemTable(itemTbl)
itemTbl = fp.getDscItemTable(itemTbl)
fp.displayData()
for item in itemTbl:
    print(f'{item.name}=>{item.count}=>{item.idxs}')

    0   1     2     3
0  I1  I2    I5  None
1  I2  I4  None  None
2  I2  I3  None  None
3  I1  I2    I4  None
4  I1  I3  None  None
5  I2  I3  None  None
6  I1  I3  None  None
7  I1  I2    I3    I5
8  I1  I2    I3  None
I2=>7=>[0 1 2 3 5 7 8]
I1=>6=>[0 3 4 6 7 8]
I3=>6=>[2 4 5 6 7 8]
I4=>2=>[1 3]
I5=>2=>[0 7]


In [765]:
def updateItemTable(basicItem,currentNode):
    basicItem.head.append(currentNode)
    return basicItem

In [766]:
def updateTree(currentNode,basicItem,maxDepth):
    if currentNode.depth < maxDepth:
        if currentNode.is_leaf:
            newNode = Node(basicItem.name,parent=currentNode,count=1,nextNode=None)
            currentNode = newNode
            basicItem = updateItemTable(basicItem,currentNode)
        else:
            found = False
            for child in currentNode.children:
                if child.name == basicItem.name:
                    child.count +=1
                    currentNode = child
                    found=True
            if not found:
                newNode = Node(basicItem.name,parent=currentNode,count=1,nextNode=None)
                currentNode = newNode
                basicItem = updateItemTable(basicItem,currentNode)
    return currentNode,basicItem

In [767]:
root = Node('null')
currentNode = root
for i,row in enumerate(fp.rawSet):
    currentNode = root
#     print(f'===={row}====')
    maxDepth = len(row[row != None])
#     print(f'{maxDepth}')
    for basicItem in itemTbl:
        mask = np.isin(basicItem.idxs,i)
        occurance = mask.sum()
        if(occurance >= 1):
            currentNode,basicItem = updateTree(currentNode,basicItem,maxDepth)

In [773]:
for basicItem in itemTbl:
    print(basicItem.head)

[Node('/null/I2', count=7, nextNode=None)]
[Node('/null/I2/I1', count=4, nextNode=None), Node('/null/I1', count=2, nextNode=None)]
[Node('/null/I2/I3', count=2, nextNode=None), Node('/null/I1/I3', count=2, nextNode=None), Node('/null/I2/I1/I3', count=2, nextNode=None)]
[Node('/null/I2/I4', count=1, nextNode=None), Node('/null/I2/I1/I4', count=1, nextNode=None)]
[Node('/null/I2/I1/I5', count=1, nextNode=None), Node('/null/I2/I1/I3/I5', count=1, nextNode=None)]


In [774]:
itemTbl[0].head[0]

Node('/null/I2', count=7, nextNode=None)

In [775]:
print(RenderTree(root))

Node('/null')
├── Node('/null/I2', count=7, nextNode=None)
│   ├── Node('/null/I2/I1', count=4, nextNode=None)
│   │   ├── Node('/null/I2/I1/I5', count=1, nextNode=None)
│   │   ├── Node('/null/I2/I1/I4', count=1, nextNode=None)
│   │   └── Node('/null/I2/I1/I3', count=2, nextNode=None)
│   │       └── Node('/null/I2/I1/I3/I5', count=1, nextNode=None)
│   ├── Node('/null/I2/I4', count=1, nextNode=None)
│   └── Node('/null/I2/I3', count=2, nextNode=None)
└── Node('/null/I1', count=2, nextNode=None)
    └── Node('/null/I1/I3', count=2, nextNode=None)


In [669]:
root = Node('null',count=1)
currentNode = root
print(fp.df)
for basicItem in filteredTbl:
    currentNode = root
    for row in fp.rawSet:
        mask = np.isin(row,basicItem.name)
        occurance = mask.sum()
        if(occurance >= 1):
            if currentNode.depth >= len(row):
                continue         
            if currentNode.is_leaf:
                newNode = Node(basicItem.name,parent=currentNode,count=1)
                currentNode = newNode
            elif basicItem.name in currentNode.children:
                for child in currentNode.children:
                    child.count += 1
                    currentNode = child
            else:
                child = currentNode.children[-1]
                newNode = Node(basicItem.name,parent=child,count=1)
                currentNode = newNode

    0   1     2     3
0  I1  I2    I5  None
1  I2  I4  None  None
2  I2  I3  None  None
3  I1  I2    I4  None
4  I1  I3  None  None
5  I2  I3  None  None
6  I1  I3  None  None
7  I1  I2    I3    I5
8  I1  I2    I3  None


In [460]:
for row in fp.rawSet:
    currentNode = root
    for basicItem.name in filteredTbl:
        mask = np.isin(row,basicItem.name)
        occurance = mask.sum()
        for item in row:
            if(occurance >= 1):
                if currentNode.depth >= len(row):
                    continue         
                if currentNode.is_leaf:
                    newNode = Node(item,parent=currentNode,count=1)
                    currentNode = newNode
                elif item in currentNode.children:
                    for child in currentNode.children:
                        child.count += 1
                        currentNode = child
                else:
                    child = currentNode.children[-1]
                    newNode = Node(item,parent=child,count=1)
                    currentNode = newNode            

In [461]:
print(RenderTree(root))

Node('/null', count=1)
└── Node('/null/I2', count=1)
    ├── Node('/null/I2/I2', count=1)
    │   └── Node('/null/I2/I2/I2', count=1)
    │       └── Node('/null/I2/I2/I2/I2', count=1)
    ├── Node('/null/I2/I1', count=1)
    │   └── Node('/null/I2/I1/I1', count=1)
    │       └── Node('/null/I2/I1/I1/I1', count=1)
    ├── Node('/null/I2/I3', count=1)
    │   └── Node('/null/I2/I3/I3', count=1)
    │       └── Node('/null/I2/I3/I3/I3', count=1)
    └── Node('/null/I2/I4', count=1)
        └── Node('/null/I2/I4/I4', count=1)


In [391]:
basicItem = filteredTbl[0].name
print(fp.df)
mask = np.isin(fp.rawSet,basicItem)
vec_mask = np.isin(mask.sum(axis=1),1)
idxs = np.where(vec_mask)
print(idxs)

    0   1     2     3
0  I1  I2    I5  None
1  I2  I4  None  None
2  I2  I3  None  None
3  I1  I2    I4  None
4  I1  I3  None  None
5  I2  I3  None  None
6  I1  I3  None  None
7  I1  I2    I3    I5
8  I1  I2    I3  None
(array([0, 1, 2, 3, 5, 7, 8], dtype=int64),)
